This notebook proves:

models load correctly

predictions work

NER + summarization integrate

ready for FastAPI / CI-CD

In [1]:
import joblib
import spacy
from transformers import pipeline


c:\Users\user\Desktop\MY_PROJECT'S\FIN_TECH\Intelligent Document Understanding (IDU)-NLP\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
clf = joblib.load("../models/text_classifier.pkl")
vectorizer = joblib.load("../models/tfidf_vectorizer.pkl")


In [3]:
nlp = spacy.load("en_core_web_sm")

summarizer = pipeline(
    "text2text-generation",
    model="t5-small",
    device=-1
)

Device set to use cpu


In [6]:
def process_document(text):
    # 1. Classification
    X = vectorizer.transform([text])
    category = clf.predict(X)[0]

    # 2. NER
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    # 3. Summarization
    summary = summarizer(
        "summarize: " + text,
        max_length=130,
        min_length=50,
        do_sample=False
    )[0]["generated_text"]

    return {
        "category": category,
        "entities": entities,
        "summary": summary
    }


In [8]:
sample_text = """
Essay writing is a vital part of English language learning. Even lower primary students are given essay writing to test their knowledge of English vocabulary and grammar! 
Apart from testing their knowledge and command of the English language, essays also help kids think deeply about the topic and express their views.
As your child keeps practising writing a cow essay in English, their creativity and English language knowledge improve. 
Also, essay writing helps them observe things more keenly, enhancing their perception, which is a valuable skill in essay writing. 
An essay, or a paragraph, about the most popular animal kids know, the cow, is a frequent essay topic for school kids. Cows are the most common animals kids see everywhere in India, so they make an easy topic to focus on for classes 1, 2, & 3 kids. """

output = process_document(sample_text)
output


Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'category': 'sport',
 'entities': [('English', 'LANGUAGE'),
  ('English', 'NORP'),
  ('English', 'LANGUAGE'),
  ('English', 'LANGUAGE'),
  ('English', 'LANGUAGE'),
  ('India', 'GPE'),
  ('1', 'CARDINAL'),
  ('2', 'CARDINAL'),
  ('3', 'CARDINAL')],
 'summary': 'essay writing is a vital part of English language learning . even lower primary students are given essay writing to test their knowledge of English vocabulary and grammar . essay writing helps kids think deeply about the topic and express their views . an essay about the most popular animal kids know, the cow, is a frequent essay topic for school kids .'}